# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [ ]:
# import numpy and pandas

import numpy as np
import pandas as pd


# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [ ]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [ ]:
# Your code here:
pokemon.head()


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [ ]:
# Your code here:

non_legendary = pokemon[pokemon['Legendary']==False]
legendary = pokemon[pokemon['Legendary']==True]

legendary.value_counts()
non_legendary.value_counts()



Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [ ]:
# Your code here:

leg_mean = legendary['Total'].mean()
leg_std = legendary['Total'].std()

non_leg_mean = non_legendary['Total'].mean()
non_leg_std = non_legendary['Total'].std()

print(leg_mean)
print(leg_std)
print(non_leg_mean)
print(non_leg_std)



The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [ ]:
# Your code here:
from scipy import stats

stats.ttest_ind(legendary['Total'],non_legendary['Total'])


What do you conclude from this test? Write your conclusions below.

In [ ]:
# Your conclusions here:

'''The p-value is very low which means that these two are probably indipendent from each other.'''

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [ ]:
# Your code here:

single_type = pokemon[pokemon['Type 2'].isnull()] 
single_type['Type 1'].value_counts()

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [ ]:
# Your code here:
water_type = single_type[single_type['Type 1']=='Water']
print(water_type['Total'].mean())
print(water_type['Total'].std())
other_type = single_type[single_type['Type 1'] != 'Water']
print(other_type['Total'].mean())
print(other_type['Total'].std())


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [ ]:
# Your code here:


stats.ttest_ind(water_type['Total'],other_type['Total'])

Write your conclusion below.

In [ ]:
# Your conclusions here:

''' H0 confirmed'''

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [ ]:
# Your code here:

pokemon.head()
stats.ttest_rel(pokemon['Attack'],pokemon['Defense'])

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

''' Reject H0'''

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [ ]:
# Your code here:
print(pokemon['Sp. Atk'].mean())
print(pokemon['Sp. Def'].mean())

stats.ttest_rel(pokemon['Sp. Atk'],pokemon['Sp. Def'])



Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:
''' Unable to reject H0'''


As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [ ]:
# Your code here:

print(stats.ttest_1samp(pokemon['Sp. Atk'],0.0))
print(stats.ttest_1samp(pokemon['Sp. Def'],0.0))    
    

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

